In [1]:
!pip install ffmpeg-python
import ffmpeg
import pandas as pd
import numpy  as np
import os

In [7]:
vid_events_cols = ['end_frame', 'end_time', 'cut']
vid_events = pd.read_csv('Shot_and_Scene/video_boundaries/tt0073486.videvents', 
                        delim_whitespace=True, header=0, names=vid_events_cols)
vid_events['shot_id'] = range(1, len(vid_events)+1)
vid_events['start_time'] = vid_events['end_time'].shift(1)
vid_events['start_frame'] = vid_events['end_frame'].shift(1)
vid_events.fillna(0, inplace=True)
vid_events.head(10)

,end_frame,end_time,cut,shot_id,start_time,start_frame
0,2060,85.919,CUT,1,0.000,0.0
1,2626,109.526,CUT,2,85.919,2060.0
2,3278,136.720,CUT,3,109.526,2626.0
3,3450,143.894,CUT,4,136.720,3278.0
4,3617,150.859,CUT,5,143.894,3450.0
5,4049,168.877,CUT,6,150.859,3617.0
6,4424,184.518,CUT,7,168.877,4049.0
7,4509,188.063,CUT,8,184.518,4424.0
8,4589,191.400,CUT,9,188.063,4509.0
9,5070,211.461,CUT,10,191.400,4589.0


In [3]:
scene_cols = ['start_shot_id', 'end_shot_id', 'valid']
scenes = pd.read_csv('Shot_and_Scene/scene_boundaries/tt0073486.scenes.gt', delim_whitespace=True, header=None, names=scene_cols)
scenes.head(10)

,start_shot_id,end_shot_id,valid
0,1,1,0
1,2,2,0
2,3,4,1
3,5,5,1
4,6,6,1
5,7,14,1
6,15,15,1
7,16,25,1
8,26,28,1
9,29,29,1


In [8]:
scene_times = scenes.join(vid_events[['shot_id', 'start_time', 'start_frame']].set_index('shot_id'), on='start_shot_id')
scene_times = scene_times.join(vid_events[['shot_id', 'end_time', 'end_frame']].set_index('shot_id'), on='end_shot_id')
scene_times['end_frame'].fillna(vid_events['end_frame'].max(), inplace=True)
scene_times['end_time'].fillna(vid_events['end_time'].max(), inplace=True)
scene_times[['start_frame', 'end_frame']] = scene_times[['start_frame', 'end_frame']].astype(int)
scene_times['scene_id'] = range(1, len(scene_times)+1)
scene_times.to_csv('scenes_defined.csv', index=False)
scene_times

,start_shot_id,end_shot_id,valid,start_time,start_frame,end_time,end_frame,scene_id
0,1,1,0,0.000,0,85.919,2060,1
1,2,2,0,85.919,2060,109.526,2626,2
2,3,4,1,109.526,2626,143.894,3450,3
3,5,5,1,143.894,3450,150.859,3617,4
4,6,6,1,150.859,3617,168.877,4049,5
...,...,...,...,...,...,...,...,...
224,1529,1536,1,7617.735,182643,7744.612,185685,225
225,1537,1537,1,7744.612,185685,7749.325,185798,226
226,1538,1545,1,7749.325,185798,7819.687,187485,227
227,1546,1553,1,7819.687,187485,7854.972,188331,228


In [11]:
scene_times.groupby(by='valid').count()

,start_shot_id,end_shot_id,start_time,start_frame,end_time,end_frame,scene_id
valid,,,,,,,
0,17,17,17,17,17,17,17
1,212,212,212,212,212,212,212


In [5]:
def saveFrameRange(file_path, start_frame, end_frame, out = "", frame_skips = 1):
    probe = ffmpeg.probe(file_path)
    width = probe['streams'][0]['width']
    file_path

    if not os.path.exists(out):
        os.makedirs(out)

    for i in range(start_frame, end_frame,frame_skips):
        ffmpeg.input(file_path, ss=i).filter('scale', width, -1).output(out+'Image' + str(i) + '.jpg', vframes=1).run()

In [105]:
movie_file = "tt0073486_One.Flew.Over.the.Cuckoos.Nest.1975.FRENCH.BRRip.x264-Wawacity.cc.mp4"
probe = ffmpeg.probe(movie_file)

frame_skips = 400
for i in scene_times['scene_id'].tolist():
    if i > 10:
        scene = scene_times[scene_times['scene_id'] == i]
        if scene['valid'].values[0] == 1:
            start_frame = scene['start_frame'].values[0]
            end_frame = scene['end_frame'].values[0]
            saveFrameRange(movie_file, start_frame, end_frame, f"scene_library/valid_scene_{i}/", frame_skips = frame_skips)
        else:
            start_frame = scene['start_frame'].values[0]
            end_frame = scene['end_frame'].values[0]
            saveFrameRange(movie_file, start_frame, end_frame, f"scene_library/invalid_scene_{i}/", frame_skips = frame_skips)

KeyboardInterrupt: 